In [73]:
import pandas as pd
from embedding import Embedding
import torch.nn as nn
import torch
import numpy as np
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import time

In [74]:
data = pd.read_csv('C:/Users/user/OneDrive/문서/dev/UROP/Crawling/data/sample_comment.csv')
token = pd.read_csv('C:/Users/user/OneDrive/문서/dev/UROP/Crawling/tokenized_0.csv')

In [75]:
df = data[['mbti','comments']]

In [76]:
df

,mbti,comments
0,ENTJ,눈빛과 기류를 보면 알지 않나용 일단 진심으로 이뻐하는 게 여기까지 느껴지네요
1,ENFJ,저도 눈빛과 기류를 보고 알수있다고 생각해요
2,ENFP,걱정하지마세요 엔프제라고 해서 다 조리있게 말하지는 않더라구요 대신 공통점이 칭찬은...
3,ENFP,네 말도 많구 말에 칭찬이 많이 차지하더라구요
4,ISTP,밀당아닌가요
...,...,...
995,INFJ,확실한건 너무좋아하면 저처럼 읽씹당해도 또보내긴 하는데요 저런 잘자라는 멘트는 좋아...
996,ISTP,후전 연락온다면 적극적으로 다시 변할것 같은데굳이 저혼자 자꾸 먼저 보내기가 싫네요...
997,INFJ,망상의동물이라 시간이 지날수록 혼자 망상하다가 마음정리할수도 있어요 지금 비유를하면...
998,ISTP,카톡 프사에도 반응하나요 제 얼굴 사진으로 바꿔볼까해서요


In [77]:
df = df.dropna().reset_index(drop=True)
token = token.dropna().reset_index(drop=True)

In [78]:
df

,mbti,comments
0,ENTJ,눈빛과 기류를 보면 알지 않나용 일단 진심으로 이뻐하는 게 여기까지 느껴지네요
1,ENFJ,저도 눈빛과 기류를 보고 알수있다고 생각해요
2,ENFP,걱정하지마세요 엔프제라고 해서 다 조리있게 말하지는 않더라구요 대신 공통점이 칭찬은...
3,ENFP,네 말도 많구 말에 칭찬이 많이 차지하더라구요
4,ISTP,밀당아닌가요
...,...,...
982,INFJ,확실한건 너무좋아하면 저처럼 읽씹당해도 또보내긴 하는데요 저런 잘자라는 멘트는 좋아...
983,ISTP,후전 연락온다면 적극적으로 다시 변할것 같은데굳이 저혼자 자꾸 먼저 보내기가 싫네요...
984,INFJ,망상의동물이라 시간이 지날수록 혼자 망상하다가 마음정리할수도 있어요 지금 비유를하면...
985,ISTP,카톡 프사에도 반응하나요 제 얼굴 사진으로 바꿔볼까해서요


In [79]:
for i in range(len(df)):
    if 'E' in df.loc[i, 'mbti']:
        df.loc[i, 'mbti'] = 0
    else:
        df.loc[i, 'mbti'] = 1

In [80]:
label = [int(i) for i in df['mbti'].values.tolist()]
print(len(label))
label_tensor = torch.LongTensor(label)
print(label_tensor)

987
tensor([0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0,
        0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1,
        0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1,
        0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1,
        1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1,
        1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
        0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 0, 0, 1, 1

In [81]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, batch_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.batch_size = batch_size
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.hidden = self.init_hidden()
        # self.embedding = nn.Embedding(vocab_size, input_size)

    def init_hidden(self):
        return (torch.zeros(1, self.batch_size, self.hidden_size),
                torch.zeros(1, self.batch_size, self.hidden_size))

    def forward(self, input):
        # print(input.shape)
        # embedded = self.embedding(input)
        lstm_out, self.hidden = self.lstm(input, self.hidden)

        # print(input.size(-1))
        # lstm_out, self.hidden = self.lstm(input.view(len(input), 1, -1), self.hidden)
        output = self.fc(lstm_out[:, -1, :])
        return output

In [91]:
for i, (inputs, labels) in enumerate(val_loader):
    # 처음 몇 개의 배치만 출력
    if i < 5:
        print(f"Batch {i+1} - Inputs: {inputs}, Labels: {labels}")
    else:
        break

Batch 1 - Inputs: tensor([[[-0.0044, -0.0299, -0.0020,  ...,  0.0155,  0.0590,  0.0415],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.1159, -0.2099,  0.4562,  ...,  0.9522,  0.7527,  1.1380],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.1163, -0.2025,  0.3932,  ...,  0.7782,  0.6422,  0.9529],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

        [[ 0.0604, -0.1497,  0.2510,  ...,  0.5808,  0.4873,  0.7201],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0616, -0.1785,  0.3326,  ...,  0.6479,  0.5227,  0.7834],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0728, -0.0544,  0.1619,  ...,  0.3537,  0.3408,  0.4485],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

        [[ 0.0390, -0.0489,  0.1151,  ...,  0.1405,  0.1149,  0.2064],
         [ 0.0615, -0.0693,

In [82]:
em = Embedding(sereis=token['comments'])
word2vec_model = em.get_embedding_model()

[I 2023-12-10 03:34:16,793] A new study created in memory with name: no-name-1ca3cf95-5dd6-4ff1-841a-ccef1068aa81
[I 2023-12-10 03:34:16,845] Trial 0 finished with value: 33904.3515625 and parameters: {'vector_size': 25, 'window': 6, 'min_count': 24, 'sg': 0}. Best is trial 0 with value: 33904.3515625.


In [83]:
# Word2Vec 모델의 크기를 기반으로 LSTM 모델 생성
input_size = word2vec_model.vector_size
hidden_size = 128
output_size = 1
batch_size = 32
# vocab_size = len(word2vec_model.wv.key_to_index)
lstm_model = LSTMModel(input_size, hidden_size, output_size, batch_size)
# print(vocab_size)

In [84]:
# 단어에 대한 0 벡터 생성
def get_word_vector(word, model):
    try:
        return model.wv[word]
    except:
        return np.zeros(input_size)

# 모든 단어에 대한 벡터 리스트 생성
# word_vectors = [get_word_vector(word, word2vec_model) for sentence in em.corpus for word in sentence]
end = 100
word_vectors = []

for sentence in em.corpus:
    temp = []
    for word in sentence[:end]:  # 문장을 end 길이로 자르기
        temp.append(get_word_vector(word, word2vec_model))

    # 패딩 추가
    padding_length = end - len(temp)
    if padding_length > 0:
        padding = [np.zeros(word2vec_model.vector_size) for _ in range(padding_length)]
        temp.extend(padding)

    word_vectors.append(temp)

In [85]:
# word_to_index = {word: index for index, word in enumerate(word2vec_model.wv.key_to_index)}

# 단어에 대한 0 벡터 생성
def get_word_vector(word, model):
    try:
        return model.wv[word]
    except:
        return np.zeros(input_size)


# 모든 단어에 대한 벡터 리스트 생성
# word_vectors = [get_word_vector(word, word2vec_model) for sentence in em.corpus for word in sentence]
end = 100
word_vectors = []

for sentence in em.corpus:
    temp = []
    for word in sentence[:end]:  # 문장을 end 길이로 자르기
        temp.append(get_word_vector(word, word2vec_model))

    # 패딩 추가
    padding_length = end - len(temp)
    if padding_length > 0:
        padding = [np.zeros(word2vec_model.vector_size) for _ in range(padding_length)]
        temp.extend(padding)

    word_vectors.append(temp)

In [86]:
# 벡터 리스트를 텐서로 변환
input_tensor = torch.tensor(word_vectors, dtype=torch.float32)
# input_tensor = torch.LongTensor(word_vectors)

In [87]:
trn_data=input_tensor[:800]
trn_label=label_tensor[:800]
val_data=input_tensor[800:]
val_label=label_tensor[800:]

In [88]:
print(trn_data.shape)
print(trn_label.shape)
print(val_data.shape)
print(val_label.shape)

torch.Size([800, 100, 25])
torch.Size([800])
torch.Size([187, 100, 25])
torch.Size([187])


In [89]:
# len(input_tensor[0])
print(input_tensor.shape)
print(input_tensor.size(-1))

torch.Size([987, 100, 25])
25


In [90]:
# LSTM 모델에 입력 전달하여 출력 받기
output = lstm_model(input_tensor)
print("LSTM 모델의 출력:", output, len(output))

RuntimeError: Expected hidden[0] size (1, 987, 128), got [1, 32, 128]

In [ ]:
# BCE 손실 함수 정의
criterion = nn.BCEWithLogitsLoss()
# Adam 옵티마이저 설정
optimizer = optim.Adam(lstm_model.parameters(), lr=0.001)

In [ ]:
class MyDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

In [ ]:
# data = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
# dataset = MyDataset(input_tensor, label_tensor)
trn_dataset = MyDataset(trn_data, trn_label)
val_dataset = MyDataset(val_data, val_label)

# batch_size = 32
train_loader = DataLoader (trn_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader (val_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
# 학습 및 평가 루프 정의
def train(model, criterion, optimizer, train_loader, epochs=5):
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:  
            # train_loader에는 입력 데이터와 레이블이 있는 데이터가 포함되어야 합니다.
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), labels.float())  
            # BCELoss를 사용하므로 레이블을 float로 변환합니다.
            
            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss / len(train_loader)}")

In [ ]:
def evaluate(model, criterion, eval_loader):
    model.eval()  # 평가 모드로 모델 설정
    total_loss = 0.0
    total_corrects = 0
    with torch.no_grad():  # 기울기 계산 비활성화
        for inputs, labels in eval_loader:
            # if inputs.size(0) == 1:
            # print(inputs.shape)
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), labels.float())

            total_loss += loss.item()
            # 예측된 클래스를 계산하고, 정확도를 측정합니다.
            # 이 부분은 사용하는 모델과 문제에 따라 다를 수 있습니다.
            preds = outputs.round()  # 예: 이진 분류의 경우
            total_corrects += torch.sum(preds == labels.data)

    # 평균 손실과 정확도 계산
    avg_loss = total_loss / len(eval_loader)
    accuracy = total_corrects.double() / len(eval_loader.dataset)

    print(f"Loss: {avg_loss}, Accuracy: {accuracy}")

    return avg_loss, accuracy

In [ ]:
def train_and_evaluate(model, criterion, optimizer, train_loader, eval_loader, epochs=5):
    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}")

        # 학습 루프
        train(model, criterion, optimizer, train_loader, epochs=1)

        # 평가 루프
        eval_loss, eval_accuracy = evaluate(model, criterion, eval_loader)

        # 에포크 결과 출력
        print(f"Validation Loss: {eval_loss}, Validation Accuracy: {eval_accuracy}")

# 모델, 손실 함수, 옵티마이저, 데이터 로더 등을 정의한 후
train_and_evaluate(lstm_model, criterion, optimizer, train_loader, val_loader, epochs=5)


Epoch 1/5


ValueError: Target size (torch.Size([32])) must be the same as input size (torch.Size([100]))